# Upload File

In [ ]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("2020_2024.jsonl", "rb"),
  purpose="batch"
)

# Create Batch

In [ ]:
client.batches.create(
  input_file_id="file-abc123",
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

# Retrieve Batch

In [ ]:
client.batches.retrieve("batch_abc123")